In [6]:
import os 
import yaml
def read_config(path):
    """
    Reads API key from a configuration file.

    This function opens a configuration file named "apikeys.yml", reads the API key for OpenAI

    Returns:
    api_key (str): The API key for the Amadeus Flights API.
    """
    
    # Get the directory of the current script
    script_dir = path

    # Construct the full path to the configuration file
    file_path = os.path.join(script_dir, "apikeys.yml")

    with open(file_path, 'r') as stream:
        configs = yaml.safe_load(stream)
        API_KEY = configs['openai']['api_key']
            
    return API_KEY
path = r"C:\Users\johna\OneDrive\Documents\api_keys"  # Change to the location of your apikeys.yml
API_KEY = read_config(path)

In [4]:
from haystack.nodes import PreProcessor
from haystack.utils import convert_files_to_docs
from haystack.document_stores import FAISSDocumentStore
from sqlalchemy import create_engine

# pre-process docs 
def preprocess_docs(doc_dir):
    all_docs = convert_files_to_docs(dir_path=doc_dir)
    preprocessor = PreProcessor(
        clean_empty_lines=True,
        clean_whitespace=True,
        clean_header_footer=False,
        split_by="word",
        split_respect_sentence_boundary=True,
        split_overlap=30, 
        split_length=100
    )
    docs = preprocessor.process(all_docs)
    print(f"n_files_input: {len(all_docs)}\nn_docs_output: {len(docs)}")
    return docs


# create FAISS
def vector_stores(docs):
    engine = create_engine('sqlite:///C:/Users/johna/anaconda3/envs/longfunctioncall_env/long_functioncall/database/database.db')  # change to your local directory
    try:
        # Attempt to drop the table
        engine.execute("DROP TABLE document")
    except Exception as e:
        # Catch any exceptions, likely due to the table not existing
        print(f"Exception occurred while trying to drop the table: {e}")
    
    document_store = FAISSDocumentStore(sql_url='sqlite:///C:/Users/johna/anaconda3/envs/longfunctioncall_env/long_functioncall/database/database.db', faiss_index_factory_str="Flat", embedding_dim=768) # change to your local directory
    document_store.write_documents(docs)
    
    return document_store

In [5]:
doc_dir = r"C:\Users\johna\anaconda3\envs\longfunctioncall_env\long_functioncall\knowledge_base"
docs = preprocess_docs(doc_dir)
document_store = vector_stores(docs)

Preprocessing: 100%|███████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.61docs/s]


n_files_input: 1
n_docs_output: 1176
Exception occurred while trying to drop the table: (sqlite3.OperationalError) no such table: document
[SQL: DROP TABLE document]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


Writing Documents: 10000it [00:02, 3442.53it/s]                                                                        


In [50]:
import openai
from haystack.nodes.base import BaseComponent
from typing import List
import json

class OpenAIFunctionCall(BaseComponent):
    outgoing_edges = 1

    def run(self, documents: List[str]):
        
        # Try to extract the content and print the first few content strings
        try:
            document_content_list = [doc.content for doc in documents]
            print("documents extracted")
            document_content = " ".join(document_content_list)
        except Exception as e:
            print("Error extracting content:", e)
            return
        functions = [
            {
                "name": "write_to_df",
                "description": "write the fund details to a dataframe",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "prr": {
                            "type": "string",
                            "description": "The FCA product reference number which will be six or seven digits"
                        },
                        "investment_objective": {
                            "type": "string",
                            "description": "You should return the investment objective of the fund. This is likely to be something like this: The Fund aims to grow your investment over t – t + delta t years"
                        },
                        "investment_policy": {
                            "type": "string",
                            "description": "Return a summary of the fund's investment policy, no more than two sentences."
                        },
                        "investment_strategy": {
                            "type": "string",
                            "description": "Return a summary of the fund's investment strategy, no more than two sentences."
                        },
                        "ESG": {
                            "type": "string",
                            "description": "Return either True, or False. True if the fund is an ESG fund, False otherwise."
                        },
                        "fund_name": {
                            "type": "string",
                            "description": "Return the name of the fund"
                        },
                    },
                },
                "required": ["prr", 
                             "investment_objective", 
                             "investment_policy",
                             "investment_strategy",
                             "ESG",
                             "fund_name"]
            }
        ]
        
        openai.api_key = API_KEY
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=[{"role": "system", "content": document_content}],
            functions=functions,
            function_call="auto",  # auto is default, but we'll be explicit
        )

        function_call_args = json.loads(response["choices"][0]["message"]["function_call"]["arguments"])
        
        return function_call_args, "output_1"

    def run_batch(self, documents: List[str]):
        # You can either process multiple documents in a batch here or simply loop over the run method
        results = []
        for document_content in document_content:
            result, _ = self.run(document_content)
            results.append(result)
        return results, "output_1"

In [51]:
# create our pipeline
from haystack import Pipeline
from haystack.nodes import EmbeddingRetriever

retriever = EmbeddingRetriever(
    document_store=document_store,
    embedding_model="sentence-transformers/all-mpnet-base-v2"
)
document_store.update_embeddings(retriever)

Batches: 100%|█████████████████████████████████████████████████████████████████████████| 37/37 [04:01<00:00,  6.54s/it]
Documents Processed: 10000 docs [04:02, 41.29 docs/s]                                                                  


In [52]:
p = Pipeline()
p.add_node(component=retriever, name="retriever", inputs=["Query"])
p.add_node(component=OpenAIFunctionCall(), name="OpenAIFunctionCall", inputs=["retriever"])
res = p.run(query="Get the details for the Global Sustain Fund")

Batches: 100%|███████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  9.80it/s]


documents extracted


In [59]:
prr = res['prr']
investment_objective = res['investment_objective']
investment_strategy = res['investment_strategy']
investment_policy = res['investment_policy']
ESG = res['ESG']
fund_name = res['fund_name']

In [73]:
import pandas as pd

def update_dataframe(prr, investment_objective, investment_strategy, investment_policy, ESG, fund_name):

    # Define the columns
    columns = ['prr', 'fund_name', 'investment_objective', 'investment_strategy', 'investment_policy', 'ESG']
    
    # Create an empty DataFrame with the specified columns
    df = pd.DataFrame(columns=columns)
    
    # Create a dictionary for the new record
    new_record = {
        'prr': prr,
        'fund_name': fund_name,
        'investment_objective': investment_objective,
        'investment_strategy': investment_strategy,
        'investment_policy': investment_policy,
        'ESG': ESG
    }

    # Check if a record with the given prr already exists
    if prr in df['prr'].values:
        # Update the existing record
        idx = df[df['prr'] == prr].index[0]
        df.loc[idx] = new_record
    else:
        # Convert the new record to a DataFrame
        new_df = pd.DataFrame([new_record])
        # Concatenate the existing DataFrame with the new record
        df = pd.concat([df, new_df], ignore_index=True)

    return df

In [75]:
df = update_dataframe(prr, investment_objective, investment_strategy, investment_policy, ESG, fund_name)
df

,prr,fund_name,investment_objective,investment_strategy,investment_policy,ESG
0,914072,Global Sustain Fund,The Fund aims to grow your investment over 5-1...,The Investment Manager applies sustainability ...,The Fund invests at least 70% of its assets in...,True
